# Assignment

## Task 2. Coral Image Classification

In [68]:
"""
Adapted from Assignment (COMP3010)
    Tanaka Chitete
    Accessed 07/10/2022

Adapted from Image Data Loaders in PyTorch
    Shivam Chandhok
    Accessed 07/10/2022
""";

In [ ]:
# Connect to my Google Drive
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!pip install matplotlib-inline
!pip install d2l==0.17.5

In [71]:
import torch
from torch import nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms as T
from d2l import torch as d2l

In [ ]:
if torch.cuda.is_available():
  torch.cuda.empty_cache()
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

vgg16 = models.vgg16(pretrained=True).to(device)

In [ ]:
torch.manual_seed(0)

## Reading the Dataset


In [74]:
normalize = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transforms = T.Compose([T.Resize((224, 224)), T.ToTensor(), normalize])

batch_size = 128

train_path = "drive/MyDrive/Code/comp3007/assignment/task2/vgg16/resources/train"
train_data = ImageFolder(root=train_path, transform=transforms)

val_path = "drive/MyDrive/Code/comp3007/assignment/task2/vgg16/resources/val"
val_data = ImageFolder(root=val_path, transform=transforms)

# Create training and validation set dataloaders
train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

## Configuring Output Dimensions

In [ ]:
def init_weights(model):
  nn.init.normal_(model.weight, std=0.01)

out_layer = nn.Linear(4096, 2)
out_layer.apply(init_weights)

In [76]:
vgg16.classifier[6] = out_layer

## Training

In [77]:
"""
Adapted from Dive into Deep Learning.
    Aston Zhang, Zack C. Lipton, Mu Li and Alex J. Smola.
    https://d2l.ai/
    Accessed 12/05/2022
"""

def train(net, train_iter, num_epochs, lr, device):
    print('training on', device)
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    timer, num_batches = d2l.Timer(), len(train_iter)
    for epoch in range(num_epochs):
        # Sum of training loss, sum of training accuracy, no. of examples
        metric = d2l.Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            timer.start()
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            if not torch.isnan(l):
                l.backward()
                optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            timer.stop()
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
        print(f'epoch {epoch + 1}, loss {train_l:.3f}, train acc {train_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec '
          f'on {str(device)}')

In [78]:
learning_rate, num_epochs = 0.05, 10
train(vgg16, train_data_loader, num_epochs, learning_rate, d2l.try_gpu())

training on cuda:0
epoch 1, loss 0.621, train acc 0.686
epoch 2, loss 0.758, train acc 0.607
epoch 3, loss 0.596, train acc 0.677
epoch 4, loss 0.548, train acc 0.741
epoch 5, loss 0.427, train acc 0.809
epoch 6, loss 0.302, train acc 0.875
epoch 7, loss 0.227, train acc 0.919
epoch 8, loss 0.231, train acc 0.904
epoch 9, loss 0.078, train acc 0.978
epoch 10, loss 0.135, train acc 0.951
553.6 examples/sec on cuda:0


## Evaluating

In [79]:
"""
Adapted from Dive into Deep Learning.
    Aston Zhang, Zack C. Lipton, Mu Li and Alex J. Smola.
    https://d2l.ai/
    Accessed 12/05/2022
"""

def evaluate(net, data_iter, device=None):
    if isinstance(net, nn.Module):
        net.eval()  # Set the model to evaluation mode
        if not device:
            device = next(iter(net.parameters())).device
    # No. of correct predictions, no. of predictions
    metric = d2l.Accumulator(2)

    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(X, list):
                # Required for BERT Fine-tuning
                X = [x.to(device) for x in X]
            else:
                X = X.to(device)
            y = y.to(device)
            metric.add(d2l.accuracy(net(X), y), y.numel())
    test_acc = metric[0] / metric[1]
    print(f'test acc {test_acc:.3f}')

In [80]:
evaluate(vgg16, val_data_loader)

test acc 0.968
